In [1]:
import pandas as pd

import pyarrow.parquet as pq
from pyarrow.parquet import ParquetFile
import pyarrow as pa 
from sklearn.utils import resample
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy as np
import datetime

# формируем выборку по юзерам, которые делали хотя бы 1 клик + по всем кампаниям где есть хотя бы 1 клик, удаляем тех, для которых в данных все нули или все единицы

In [2]:
parquet_file = pq.ParquetFile('../../input_data/train.parquet')

batch_size = 115_000_000
rows = next(parquet_file.iter_batches(batch_size=batch_size)) 
df = rows.to_pandas()  
rows=0
df

,user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main,target
0,2853707,3352,3,3075,2024-09-17,6,True,0
1,2537244,4029,2,3260,2024-09-17,8,True,0
2,63033,1578,3,1109,2024-09-17,6,True,0
3,164702,3434,1,1079,2024-09-17,7,True,0
4,2802905,2208,3,3576,2024-09-17,6,True,0
...,...,...,...,...,...,...,...,...
114741030,2670849,3904,2,3275,2024-09-08,8,True,0
114741031,3187004,2142,2,238,2024-09-08,8,True,0
114741032,821310,2414,2,2654,2024-09-08,8,True,0
114741033,2282258,3120,2,551,2024-09-08,5,False,1


In [3]:
user_unique = df.user_id.unique()
len(user_unique)

3263615

In [4]:
user_0 = df.groupby('user_id').target.sum()
user_0 = user_0[user_0==0].index
user_0

Index([      1,       2,       3,       4,       5,       6,       7,       8,
             9,      10,
       ...
       3263611, 3263612, 3263613, 3263614, 3263615, 3263617, 3263618, 3263620,
       3263621, 3263622],
      dtype='int64', name='user_id', length=2864504)

In [5]:
campaign_0 = df.groupby('adv_campaign_id').target.sum()
campaign_0 = campaign_0[campaign_0==0].index
campaign_0

Index([   7,   10,   13,   16,   21,   44,   67,   77,   84,  113,
       ...
       4013, 4025, 4042, 4065, 4098, 4154, 4156, 4161, 4171, 4192],
      dtype='int64', name='adv_campaign_id', length=265)

In [6]:
user_1 = df.groupby('user_id').target.sum()
user_1_ = df.groupby('user_id').target.count()
user_1 = pd.concat([user_1.rename('sum'), user_1_.rename('count')], axis=1)

user_1['equal'] = np.where(user_1['sum'] == user_1['count'], 1, 0)
user_1 = user_1[user_1.equal==1].index
user_1

Index([    545,    1586,    2307,    3159,    3519,    4196,    6817,    7541,
          8830,   11926,
       ...
       3253016, 3253076, 3255739, 3256201, 3256380, 3256877, 3256963, 3258326,
       3260192, 3263256],
      dtype='int64', name='user_id', length=3217)

In [7]:
df = df[~df.user_id.isin(user_0)]
df = df[~df.user_id.isin(user_1)]
df = df[~df.user_id.isin(campaign_0)]
df

,user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main,target
0,2853707,3352,3,3075,2024-09-17,6,True,0
1,2537244,4029,2,3260,2024-09-17,8,True,0
2,63033,1578,3,1109,2024-09-17,6,True,0
3,164702,3434,1,1079,2024-09-17,7,True,0
7,683880,1848,2,3205,2024-09-17,8,True,0
...,...,...,...,...,...,...,...,...
114741028,336444,3729,2,3305,2024-09-08,5,False,0
114741031,3187004,2142,2,238,2024-09-08,8,True,0
114741032,821310,2414,2,2654,2024-09-08,8,True,0
114741033,2282258,3120,2,551,2024-09-08,5,False,1


In [8]:
class1 = df[df.target==1]
class0 = resample(df[df.target==0], replace=False, n_samples=df.target.value_counts()[1] * 14, random_state=0)
data = pd.concat([class0, class1], axis=0)
data

,user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main,target
47689269,2414445,3542,2,4140,2024-09-22,8,True,0
56807334,673480,3858,3,2606,2024-09-13,6,True,0
71047004,1817385,2751,2,2933,2024-09-20,8,True,0
103259411,465888,989,2,2066,2024-09-19,8,True,0
85625649,2868868,314,2,2603,2024-09-06,8,True,0
...,...,...,...,...,...,...,...,...
114740487,376997,3910,2,902,2024-09-08,8,True,1
114740659,521919,3711,2,2599,2024-09-08,8,True,1
114740837,1445740,1197,2,3665,2024-09-08,8,True,1
114740904,1042451,2551,3,2563,2024-09-08,6,True,1


In [9]:
test = data.groupby('user_id').target.count()
idx = test[test<2].index
idx

Index([    107,     151,     214,     332,     415,     476,     556,     573,
           984,    1002,
       ...
       3262435, 3262439, 3262581, 3262744, 3262819, 3262849, 3263225, 3263313,
       3263336, 3263565],
      dtype='int64', name='user_id', length=14311)

In [10]:
class0_ = df[df.target==0]
class0_ = class0_[class0_.user_id.isin(idx)].sort_values(by=['user_id'])

In [11]:
data = pd.concat([data, class0_], axis=0)
test = data.groupby('user_id').target.count()
idx = test[test<2].index
idx

Index([], dtype='int64', name='user_id')

In [12]:
test = data.groupby('user_id').target.count()
test[test<2]

Series([], Name: target, dtype: int64)

In [13]:
data

,user_id,adv_campaign_id,platform_id,adv_creative_id,event_date,banner_code,is_main,target
47689269,2414445,3542,2,4140,2024-09-22,8,True,0
56807334,673480,3858,3,2606,2024-09-13,6,True,0
71047004,1817385,2751,2,2933,2024-09-20,8,True,0
103259411,465888,989,2,2066,2024-09-19,8,True,0
85625649,2868868,314,2,2603,2024-09-06,8,True,0
...,...,...,...,...,...,...,...,...
110967571,3263336,3736,2,2148,2024-09-21,8,True,0
110976478,3263336,3023,2,2575,2024-09-21,8,True,0
111019322,3263336,229,2,15,2024-09-21,8,True,0
110109115,3263336,3179,2,1840,2024-09-20,5,False,0


In [14]:
tl = TomekLinks(sampling_strategy='majority')
X, y = tl.fit_resample(data.drop(columns=['target', 'event_date']), data.target)

res2 = data.event_date.iloc[tl.sample_indices_].reset_index().drop(columns=['index'])
X2 = pd.concat([pd.concat([X, res2], axis=1), y], axis=1) 

X2

,user_id,adv_campaign_id,platform_id,adv_creative_id,banner_code,is_main,event_date,target
0,2414445,3542,2,4140,8,True,2024-09-22,0
1,673480,3858,3,2606,6,True,2024-09-13,0
2,1817385,2751,2,2933,8,True,2024-09-20,0
3,465888,989,2,2066,8,True,2024-09-19,0
4,2868868,314,2,2603,8,True,2024-09-06,0
...,...,...,...,...,...,...,...,...
9022671,3263336,3736,2,2148,8,True,2024-09-21,0
9022672,3263336,3023,2,2575,8,True,2024-09-21,0
9022673,3263336,229,2,15,8,True,2024-09-21,0
9022674,3263336,3179,2,1840,5,False,2024-09-20,0


In [15]:
test = X2.groupby('user_id').target.count()
idx = test[test<2].index
X2_ = X2[~(X2.user_id.isin(idx))]
X2_

,user_id,adv_campaign_id,platform_id,adv_creative_id,banner_code,is_main,event_date,target
0,2414445,3542,2,4140,8,True,2024-09-22,0
1,673480,3858,3,2606,6,True,2024-09-13,0
2,1817385,2751,2,2933,8,True,2024-09-20,0
3,465888,989,2,2066,8,True,2024-09-19,0
4,2868868,314,2,2603,8,True,2024-09-06,0
...,...,...,...,...,...,...,...,...
9022671,3263336,3736,2,2148,8,True,2024-09-21,0
9022672,3263336,3023,2,2575,8,True,2024-09-21,0
9022673,3263336,229,2,15,8,True,2024-09-21,0
9022674,3263336,3179,2,1840,5,False,2024-09-20,0


In [16]:
X2_.to_csv('smart_resample_f.csv', sep=',', index=False)